In [6]:
from sqlalchemy import create_engine
import pandas as pd

In [3]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [5]:
table_names = engine.table_names()
table_names

C:\Users\emily\AppData\Local\Temp\ipykernel_47624\4286363722.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


['Bookings', 'Facilities', 'Members']

In [27]:
query = """
SELECT name, SUM(revenue) AS total_revenue
FROM 
    (SELECT name, 
    CASE 
        WHEN memid = 0 THEN guestcost * slots
        ELSE membercost * slots
        END AS revenue
    FROM Bookings as b
    JOIN Facilities as f
    USING (facid)) AS x
GROUP BY name
HAVING total_revenue < 1000
ORDER BY total_revenue
"""
df = pd.read_sql_query(query, engine)
df

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [50]:
query = """
SELECT m1.surname || ', '|| m1.firstname AS member_name, m2.surname || ', ' || m2.firstname AS recommender_name
FROM Members as m1
RIGHT JOIN Members as m2
ON m1.recommendedby = m2.memid
WHERE m1.recommendedby IS NOT NULL
ORDER BY member_name
"""
df = pd.read_sql_query(query, engine)
df

,member_name,recommender_name
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


In [54]:
# Just out of curiosity
query = """
SELECT COUNT(*), m2.surname || ', ' || m2.firstname AS recommender_name
FROM Members as m1
RIGHT JOIN Members as m2
ON m1.recommendedby = m2.memid
WHERE m1.recommendedby IS NOT NULL
GROUP BY m2.surname, m2.firstname
ORDER BY recommender_name
"""
df = pd.read_sql_query(query, engine)
df

,COUNT(*),recommender_name
0,1,"Bader, Florence"
1,1,"Baker, Timothy"
2,1,"Butters, Gerald"
3,2,"Farrell, Jemima"
4,1,"Genting, Matthew"
5,1,"Jones, David"
6,2,"Joplette, Janice"
7,1,"Purview, Millicent"
8,1,"Rownam, Tim"
9,5,"Smith, Darren"


In [73]:
query = """
SELECT name, COUNT(*) AS member_usage 
FROM Bookings
JOIN Facilities
USING(facid)
WHERE memid > 0
GROUP BY name
"""
df = pd.read_sql_query(query, engine)
df

,name,member_usage
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


In [82]:
query = """
SELECT STRFTIME('%m', starttime) AS month, COUNT(*) AS member_usage 
FROM Bookings
JOIN Facilities
USING(facid)
WHERE memid > 0
GROUP BY STRFTIME('%m', starttime)
"""
df = pd.read_sql_query(query, engine)
df

,month,member_usage
0,07,480
1,08,1168
2,09,1512


In [80]:
query = """
SELECT MAX(starttime), MIN(starttime)
FROM Bookings;
"""
df = pd.read_sql_query(query, engine)
df

,MAX(starttime),MIN(starttime)
0,2012-09-30 19:30:00,2012-07-03 08:00:00


In [14]:
bookings_sample = con.execute('SELECT * FROM Bookings LIMIT 5')
pd.DataFrame(bookings_sample.fetchall())

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [13]:
facilities_sample = con.execute('SELECT * FROM Facilities LIMIT 5')
pd.DataFrame(facilities_sample.fetchall())

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [15]:
members_sample = con.execute('SELECT * FROM Members LIMIT 5')
pd.DataFrame(members_sample.fetchall())

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [16]:
con.close()